In [ ]:
#Loading of the dataset for the study
import pandas as pd
import pyarrow as pa 
import pyarrow.parquet as pq

DfLabelledTimeSeriesCleaned=pd.read_parquet('F:/Data_BachelorHES/DataSet_CNC/CNC_Labelled_Data_Cleaned.parquet', engine='pyarrow')

In [ ]:
# Fonction pour échapper les caractères spéciaux dans les tags
def sanitize_tag(value):
    if isinstance(value, str):
        return value.replace(' ', '\ ').replace(',', '\,').replace('=', '\=')
    return value

In [ ]:
from tqdm import tqdm
import numpy as np

# Initialize tqdm to show progress bar
totalRows = len(DfLabelledTimeSeriesCleaned)

# Conversion au format line protocol
prevTStamp=1731747600000000000+5702400000000000 #21 janvier 2025 9h00

with open('F:/Data_BachelorHES/DataSet_CNC/influxData.lp', 'w', newline='') as f:
    for _, row in tqdm(DfLabelledTimeSeriesCleaned.iterrows(), total=totalRows, desc="Écriture des données"):
        
        progLine=sanitize_tag(row['Label_Program_Line'])
        opName=sanitize_tag(row['Label_Op_Name'])
        toolName=sanitize_tag(row['Label_Tool_Name'])
        
        lines=[]
        for i in range(len(row['Sound'])):
            
            timeStamp=prevTStamp+i*20000
            
            # Convertir les mesures en int16 avec NumPy
            sound = np.int16(row['Sound'][i])
            accx = np.int16(row['Accel X'][i])
            accy = np.int16(row['Accel Y'][i])
            accz = np.int16(row['Accel Z'][i])
            
            line = f"cnc,frame={row['frame']},pNb={row['Op_Nb']},Op={opName},tool={toolName} Snd={sound}i,AcX={accx}i,AcY={accy}i,AcZ={accz}i {timeStamp}"
            lines.append(line)
        
        f.write('\n'.join(lines)+'\n')
        prevTStamp=prevTStamp+len(row['Sound'])*20000

In [2]:
from tqdm import tqdm
import os 

inFile='F:/Data_BachelorHES/DataSet_CNC/influxData.lp'
outPath='F:/Data_BachelorHES/DataSet_CNC/lpFiles/'
j=0
totLines=459406390
block=5000
with open(inFile, 'r', newline='') as infile:
    print("opening file")
    buffer=[]
    for idx, line in tqdm(enumerate(infile), total=totLines, desc="Processing"):
        buffer.append(line)
        if (idx+1)%block==0:
            outFile=os.path.join(outPath, f"data_{j}.lp")
            j+=1
            with open(outFile, 'w', newline='') as outfile:
                for i in buffer:
                    outfile.write(i)
            buffer=[]
    if buffer :
        outFile=os.path.join(outPath, f"data_{j}.lp")
        with open(outFile, 'w', newline='') as outfile:
            for i in buffer:
                outfile.write(i)


opening file


Processing: 100%|██████████| 459406390/459406390 [2:32:26<00:00, 50229.48it/s]  
